In [2]:
# read the data and turn it into a dataframe
#imports"
from pathlib import Path
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import PandasTools
import pandas as pd
import os

# Create a Path object for the current directory
current_directory = Path.cwd()
print("Current Directory:", current_directory.resolve())

# Creating a Path object for an example file that does not yet exist
example_file_path = current_directory / "database.csv"

# Reading the contents of the file to test if we have well acess to our data
if example_file_path.exists():
    with example_file_path.open("r") as file:
        first_line = file.readline()
        print(first_line)
else:
    print("The file does not exist.")

dataFrame= pd.read_csv("database.csv")

Current Directory: /Users/Travail/git/Amazing-Project
Reactant 1;Reactant 2;Product 1;Product 2;Product 3;yield;reaction_smarts



In [326]:
# Remove atome mapping numbers from smiles
import re

def remove_atom_mapping(smiles):
    """
    Remove atom mapping numbers from a SMILES-like notation.
    Args:
    - smiles (str): The SMILES-like notation with atom mapping numbers.
    Returns:
    - smiles_without_mapping (str): The SMILES notation without atom mapping numbers.
    """
    # Define a regular expression pattern to match atom mapping numbers
    pattern = r':\d+'
    smiles_without_mapping = re.sub(pattern, '', smiles)
    return smiles_without_mapping

In [327]:
# give smiles

import re

def Smiles(smiles):
    """
    Remove atom mapping numbers, brackets, 'H' characters, and empty parentheses from a SMILES-like notation.
    Args:
    - smiles (str): The SMILES-like notation with atom mapping numbers, brackets, 'H' characters, and empty parentheses.
    Returns:
    - smiles_cleaned (str): The cleaned SMILES notation.
    """
    # Define a regular expression pattern to match atom mapping numbers, brackets, 'H' characters, and empty parentheses
    pattern = r'[:\[\]\ H@\/]'
    # Use re.sub() to remove the specified characters
    smiles_cleaned = re.sub(pattern, '', smiles)
    # Remove empty parentheses only
    smiles_cleaned = re.sub(r'\(\)', '', smiles_cleaned)
    return smiles_cleaned

def remove_back_slash(smiles):
    """
    Remove forward slash '(\)' from a string.
    Args:
    - smiles (str): The input string.
    Returns:
    - cleaned_str (str): The string with forward slash removed.
    """
    return smiles.replace(r'(\)', '')

def remove_empty_reactants_products(smiles):
    """
    Remove forward slash '.-' from a string.
    Args:
    - smiles (str): The input string.
    Returns:
    - cleaned_str (str): The string with forward slash removed.
    """
    return smiles.replace(r'.-', '')

def convert_X(smiles):
    """
    Convert "X-" elements where X can be "Br", "Cl", or "F" into "[X-]" in a SMILES-like notation.
    Args:
    - smiles (str): The SMILES-like notation.
    Returns:
    - cleaned_smiles (str): The cleaned SMILES notation.
    """
    # Define the elements you want to convert
    elements = ['Br', 'Cl', 'F']
    # Iterate over each element
    for element in elements:
        # Replace "X-" with "[X-]"
        smiles = smiles.replace(element + '-', '[' + element + '-]')
    return smiles

def transform_anions(smiles):
    """
    Transform the input string by replacing '(N+(O-)O)' with 'NO'.
    Args:
    - input_string (str): The input string to be transformed.
    Returns:
    - transformed_string (str): The transformed string.
    """
    transformed_string = re.sub(r'N\+\(O-\)', '([N+][O-])', smiles)
    return transformed_string


In [4]:
# remove mapping from the Data Frame
for column in dataFrame.columns:
    # Apply the remove_atom_mapping function to each column
    dataFrame[column] = dataFrame[column].apply(remove_atom_mapping)
    dataFrame[column] = dataFrame[column].apply(remove_back_slash)
    dataFrame[column] = dataFrame[column].apply(Smiles)
    dataFrame[column] = dataFrame[column].apply(convert_X)
    dataFrame[column] = dataFrame[column].apply(transform_anions)
    dataFrame[column] = dataFrame[column].apply(remove_empty_reactants_products)

NameError: name 'remove_atom_mapping' is not defined

In [5]:
# Split the column values at each ";" and expand them into separate columns
split_dataFrame = dataFrame['Reactant 1;Reactant 2;Product 1;Product 2;Product 3;yield;reaction_smarts'].str.split(';', expand=True)

# Rename the columns
split_dataFrame.columns = ['Reactant 1', 'Reactant 2', 'Product 1', 'Product 2', 'Product 3', 'yield', "reaction_smarts"]

In [330]:
import pandas as pd
from rdkit import Chem
from contextlib import redirect_stdout, redirect_stderr
import os

# Define a null device depending on the operating system
if os.name == 'posix':  # Unix-like systems
    NULL_DEVICE = '/dev/null'
elif os.name == 'nt':  # Windows
    NULL_DEVICE = 'nul'
else:
    raise OSError("Unsupported operating system")

def is_valid_smiles(smiles):
    """
    Check if a SMILES string is valid.
    Args:
    - smiles (str): The SMILES string to check.
    Returns:
    - True if the SMILES string is valid, False otherwise.
    """
    try:
        with redirect_stdout(open(NULL_DEVICE, 'w')), redirect_stderr(open(NULL_DEVICE, 'w')):
            mol = Chem.MolFromSmiles(smiles)
        if mol is None:
            return False
    except Exception as e:
        # Ignore and consider the row as valid if the specific SMILES parse error occurs
        if "[12:01:51] SMILES Parse Error: Failed parsing SMILES '-'" in str(e):
            return True
        else:
            return False
    return True

def filter_valid_smiles(dataframe):
    """
    Filter rows containing valid SMILES molecules from the first 5 columns of a DataFrame.
    Args:
    - dataframe (pd.DataFrame): The DataFrame to filter.
    Returns:
    - A new DataFrame containing only the rows with valid SMILES molecules.
    """
    valid_rows = []
    for index, row in dataframe.iterrows():
        valid_smiles_in_row = True
        for column in row[:5]:  # Iterate through the first 5 columns
            if not is_valid_smiles(column):
                valid_smiles_in_row = False
                break
        if valid_smiles_in_row:
            valid_rows.append(row)
    return pd.DataFrame(valid_rows, columns=dataframe.columns)

# Example usage:
# Assuming your original DataFrame is named "original_data"
valid_smiles_df = filter_valid_smiles(split_dataFrame)

# Print the resulting DataFrame
print(valid_smiles_df)



     Reactant 1               Reactant 2 Product 1                Product 2  \
0         [Br-]           BrC(C(C#N)C#N)     [Br-]           BrC(C(C#N)C#N)   
1         [Br-]          BrC(C(C#N))(C)C     [Br-]          BrC(C(C#N))(C)C   
2         [Br-]            BrC(CC#N)(C)C     [Br-]            BrC(CC#N)(C)C   
3         [Br-]                BrC(C)(C)     [Br-]                BrC(C)(C)   
4         [Br-]                  BrC(C)C     [Br-]                  BrC(C)C   
...         ...                      ...       ...                      ...   
3115       [F-]   FC(C(N)([N+][O-])O)(C)      [F-]   FC(C(N)([N+][O-])O)(C)   
3116       [F-]  FC(C(N)([N+][O-])O)(C)N      [F-]  FC(C(N)([N+][O-])O)(C)N   
3117       [F-]   FC(C(N)([N+][O-])O)C#N      [F-]   FC(C(N)([N+][O-])O)C#N   
3118       [F-]     FC(C(N)([N+][O-])O)C      [F-]     FC(C(N)([N+][O-])O)C   
3119       [F-]     FC(C(N)([N+][O-])O)N      [F-]     FC(C(N)([N+][O-])O)N   

     Product 3               yield  \
0            

In [331]:
# Access the third column and get its first element
element = split_dataFrame.iloc[0, 2]

# Print the element and its type
print("First element of the third column:", element)
print("Type of the element:", type(element))

First element of the third column: [Br-]
Type of the element: <class 'str'>


In [333]:
def main():
    
    
    while True:
        input("Press Enter to get random products or type 'exit' to quit: ")
        filtered_columns = filtered_columns = [pd.Series(valid_smiles_df[col].dropna().unique()) for col in ["Product 1", "Product 2"]]
        selected_columns = pd.concat(filtered_columns, axis=0)
        products = selected_columns.sample(n=1)
        print("Randomly selected products:", products)

        
        choice = input("Do you want to continue? (yes/no): ").strip().lower()
        if choice == "no":
            print("Exiting...")
            break
            
if __name__ == "__main__":
    main()

Press Enter to get random products or type 'exit' to quit:  no


Randomly selected products: 111    ClC(C(C#N)C#N)
dtype: object


Do you want to continue? (yes/no):  no


Exiting...


In [336]:
# Prompt the user to enter the molecule that he wants to form
print ("you can enter the name of the molecule or its SMILES, the racemic configuration is available. Only halogen molecules are taken in account in the data base")
string_input_mol = input("Enter the molecule that you want to form: ")

you can enter the name of the molecule or its SMILES, the racemic configuration is available. Only halogen molecules are taken in account in the data base


Enter the molecule that you want to form:  ClC(C(C#N)C#N)


In [337]:
# function to determine if the user enters the SMILES notation or the usual name of the molecule
from rdkit import Chem

def is_smiles(smiles):
    """
    Check if a string represents a valid SMILES notation.
    Args:
    - smiles (str): The string to check.
    Returns:
    - is_valid (bool): True if the string is a valid SMILES notation, False otherwise.
    """
    if Chem.MolFromSmiles(smiles)== None:
        return False
    else:
        return True

# Check if the entered string is a valid SMILES notation
if is_smiles(string_input_mol):
    print(f"'{string_input_mol}' is a valid SMILES notation.")
else:
    print(f"'{string_input_mol}' is not a valid SMILES notation.")

'ClC(C(C#N)C#N)' is a valid SMILES notation.


In [338]:
!pip install pubchempy

In [339]:
# function to convert name into SMILES

import pubchempy as pcp

def name_to_smiles(molecule_name):
    """
    Convert a molecule name to a SMILES notation using PubChemPy's PubChem database.
    Args:
    - molecule_name (str): The name of the molecule.
    Returns:
    - smiles (str): The SMILES notation of the molecule, or None if retrieval fails.
    """
    try:
        compound = pcp.get_compounds(molecule_name, 'name')
        if compound:
            return compound[0].canonical_smiles
        else:
            print("Error: Unable to retrieve molecule information. Please try with the SMILE of the molecule")
            return None
    except:
        print("Error: Unable to retrieve molecule information. Please try again")
        return None

In [340]:
# Call the function to convert the molecule name to SMILES notation
if is_smiles(string_input_mol)== False:
    string_input_mol = name_to_smiles(string_input_mol)
    if string_input_mol:
        print(f"SMILES notation for {string_input_mol}: {string_input_mol}")
else:
    print (string_input_mol)

ClC(C(C#N)C#N)


In [341]:
# Function that returns the row of the elements formation reaction
def compare_molecule_with_data(element):
    return ''.join(element.split()).lower() == ''.join(string_input_mol.split()).lower()

# Initialize a variable to store the row number
found_row_number = None
rows=[]
# Iterate over the last three columns of the DataFrame
for column_name in valid_smiles_df.columns[-5:-1]:
    column = valid_smiles_df[column_name]
    # Check if any element in the column matches the input molecule
    for index, value in column.items():
        if compare_molecule_with_data(value):
            # Store the row number where the molecule is found
            found_row_number = index
            rows.append(found_row_number)
    else:
        continue  # Continue to the next column if molecule not found in current column

# Check if the molecule was found and return the row number
if found_row_number is not None:
   print(rows)
else:
    print("The product is not in the database")


[406, 1299]


In [344]:
"""
This code prints the yield and shows the image of the reaction corresponding
"""
# Convert the 'Yield' column to float
valid_smiles_df['yield'] = valid_smiles_df['yield'].astype(float)

# Assuming 'rows_to_compare' is a list containing the row numbers to compare

# Subset the DataFrame to include only the specified rows
subset_df = valid_smiles_df.iloc[rows]
print(subset_df)

# Find the index of the row with the highest yield
max_yield_row_index = subset_df['yield'].idxmax()

# Retrieve the row with the highest yield
max_yield_row = valid_smiles_df.loc[max_yield_row_index]

print("Row with the highest yield:")
print(max_yield_row)

# print the reaction
from rdkit import Chem
from rdkit.Chem import Draw
from rdkit.Chem import AllChem

# Create a reaction SMARTS string
reaction_smarts = split_dataFrame.iloc[max_yield_row_index, 6]
# Parse the reaction SMARTS string

reaction = AllChem.ReactionFromSmarts(reaction_smarts)

# Draw the reaction
reaction_image = Draw.ReactionToImage(reaction)

# Save or display the image
reaction_image.show()


           Reactant 1 Reactant 2 Product 1       Product 2 Product 3  \
572   BrC(C(C#N)C#N)C       [F-]     [Br-]  FC(C(C#N)C#N)C             
2420      ClC(C(N)N)N       [F-]     [Cl-]      FC(C(N)N)N             

          yield                             reaction_smarts  
572    4.097819  BrC(C(C#N)C#N)C.[F-]>>[Br-].FC(C(C#N)C#N)C  
2420  31.283218          ClC(C(N)N)N.[F-]>>[Cl-].FC(C(N)N)N  
Row with the highest yield:
Reactant 1                                ClC(C(N)N)N
Reactant 2                                       [F-]
Product 1                                       [Cl-]
Product 2                                  FC(C(N)N)N
Product 3                                            
yield                                       31.283218
reaction_smarts    ClC(C(N)N)N.[F-]>>[Cl-].FC(C(N)N)N
Name: 2420, dtype: object


In [307]:
from rdkit import Chem
from rdkit.Chem import Draw

# SMILES representation of the molecule
smiles = "FC(C)(([N+][O-])O)N"

# Create a molecule object from the SMILES string
mol = Chem.MolFromSmiles(smiles)

# Check if the molecule object is not None (i.e., if the SMILES string is valid)
if mol is not None:
    # Draw and display the molecule
    img = Draw.MolToImage(mol)
    img.show()
else:
    print("Invalid SMILES string")

Invalid SMILES string


[12:04:17] SMILES Parse Error: syntax error while parsing: FC(C)(([N+][O-])O)N
[12:04:17] SMILES Parse Error: Failed parsing SMILES 'FC(C)(([N+][O-])O)N' for input: 'FC(C)(([N+][O-])O)N'
